In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import openai
import streamlit as st

In [ ]:

df_patents = pd.read_csv('G:\\Meu Drive\\Colab Notebooks\\Sirius\\Projeto Final\\resultados_classificados.csv')
df_subsetores = pd.read_excel('G:\\Meu Drive\\Colab Notebooks\\Sirius\\Projeto Final\\Definição de Subsetores.xlsx')

In [ ]:
# 2. Pré-processamento dos textos

def preprocess_text(text):
    text = str(text).lower()
    text = text.replace('\n', ' ')
    return text

df_patents['processed_abstract'] = df_patents['abstract'].apply(preprocess_text)

In [ ]:
# 3. Gerar embeddings com BERT

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def gerar_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

df_patents['bert_embedding'] = df_patents['processed_abstract'].apply(gerar_embedding)

df_subsetores['bert_embedding'] = df_subsetores['KeyWords'].apply(lambda keywords: gerar_embedding(' '.join(str(keywords).split(','))))

In [ ]:
print(df_patents.columns)

Index(['patent_id', 'Ecosystem', 'class_IPC_concat', 'abstract',
       'processed_abstract', 'predicted_subsetor', 'bert_embedding'],
      dtype='object')


In [ ]:
#codigo ajustado do 4 ao 6

import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

# Preparar dados para classificação
X = np.array(df_patents['bert_embedding'].tolist())  # Converter lista de embeddings para np.array
y = df_patents['predicted_subsetor'].astype('category').cat.codes

y = to_categorical(y)

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Verificar shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Criar o modelo de Deep Learning
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Usa X_train.shape[1] para input_dim
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

X_train shape: (23508, 768)
y_train shape: (23508, 16)
X_test shape: (5877, 768)
y_test shape: (5877, 16)
Epoch 1/10


C:\Users\joao_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


735/735 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3772 - loss: 1.9047 - val_accuracy: 0.4322 - val_loss: 1.6680
Epoch 2/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4454 - loss: 1.6672 - val_accuracy: 0.4866 - val_loss: 1.5613
Epoch 3/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4756 - loss: 1.5890 - val_accuracy: 0.4710 - val_loss: 1.5610
Epoch 4/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4858 - loss: 1.5566 - val_accuracy: 0.4809 - val_loss: 1.5168
Epoch 5/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5056 - loss: 1.4924 - val_accuracy: 0.5103 - val_loss: 1.4563
Epoch 6/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5137 - loss: 1.4477 - val_accuracy: 0.5185 - val_loss: 1.4393
Epoch 7/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5182 - loss: 1.4359 - val_accuracy: 0.5334 - val_loss: 1.3943
Epoch 8/10
735/735 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5272 - loss: 1.4255 - val_accuracy: 0.5288 - val_

In [ ]:
# 4. Preparar dados para classificação

X = list(df_patents['bert_embedding'])
y = df_patents['predicted_subsetor'].astype('category').cat.codes

y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# 5. Criar o modelo de Deep Learning

model = Sequential()
model.add(Dense(128, input_dim=len(X_train[0]), activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

C:\Users\joao_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

In [ ]:
# 6. Treinar o modelo

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# 7. Implementação de Explicação com GPT-4

from openai import OpenAI

# Inicialize o cliente da OpenAI
client = OpenAI(api_key="xxxxxxxxxxxx")  # Substitua por sua chave real

def gerar_explicacao(abstract, subsetores_relevantes):
    subsetores_info = '\n'.join([f"Subsetor: {row['Subsetores']}, Palavras-chave: {row['KeyWords']}" for _, row in subsetores_relevantes.iterrows()])

    prompt = f"""
    Você é um especialista em patentes. Abaixo está o resumo de uma patente e os subsetores relevantes com base nas palavras-chave.

    Resumo da patente: {abstract}

    Subsetores relevantes:
    {subsetores_info}

    Por favor, categorize essa patente no subsetor mais apropriado e forneça uma explicação detalhada sobre o motivo dessa escolha.
    """

    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",  # Ou "gpt-3.5-turbo"
        messages=[
            {"role": "system", "content": "Você é um especialista em patentes."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=400
    )

    return response.choices[0].message.content.strip()

# Exemplo de uso
test_abstract = df_patents['processed_abstract'].iloc[0]
subsetores_relevantes = df_subsetores
explicacao = gerar_explicacao(test_abstract, subsetores_relevantes)
print(explicacao)

A patente em questão se enquadra mais apropriadamente no subsetor **Fintech**, apesar de haver aspectos que tangenciam outros subsetores como "Cybersecurity" e "Industry 4.0". Permita-me explicar o porquê dessa escolha:

1. **Alinhamento com o Foco do Subsetor**: O resumo descreve um método e aparelho para fornecer redundância em um caixa eletrônico (ATM). Isso se encaixa de forma natural dentro do subsetor Fintech, que abrange inovações tecnológicas aplicadas ao setor financeiro, incluindo a otimização e segurança de operações bancárias e financeiras. Caixas eletrônicos são fundamentais para as operações do dia a dia dos bancos, que são instituições-chave no setor financeiro.

2. **Aspect


In [ ]:
print(df_patents.columns)  # Lista todas as colunas do DataFrame


Index(['patent_id', 'Ecosystem', 'class_IPC_concat', 'abstract',
       'processed_abstract', 'predicted_subsetor', 'bert_embedding'],
      dtype='object')


In [ ]:
# Supondo que seu DataFrame seja chamado df_patents
df_patents.to_csv('df_patents.csv', index=False)

In [ ]:
!streamlit run seu_script.py & npx localtunnel --port 8501


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: seu_script.py
'npx' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [ ]:
# 9. Dashboard com Streamlit

def streamlit_dashboard():
    st.title("Inteligência de Patentes com IA")
    selected_patent = st.selectbox("Selecione uma patente", df_patents['patent_id'])
    selected_row = df_patents[df_patents['patent_id'] == selected_patent]
    st.write("**Resumo:**", selected_row['processed_abstract'].values[0])
    st.write("**Subsetor Previsto:**", selected_row['predicted_subsetor'].values[0])
    st.write("**Explicação:**", gerar_explicacao(selected_row['processed_abstract'].values[0], df_subsetores))

if __name__ == "__main__":
    streamlit_dashboard()

2025-01-30 22:16:31.898 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-30 22:16:31.899 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-30 22:16:31.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-30 22:16:31.900 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-30 22:16:31.905 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-30 22:16:31.910 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-30 22:16:31.914 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-30 22:16:31.915 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar